In [178]:
from mongodb_lib import *
import yaml
import pandas as pd
from collections import defaultdict
from openai_handlers import *
import re
from tqdm import tqdm
import ast

In [179]:
config_infra = yaml.load(open("../infra-config-pipeline.yaml"), Loader=yaml.FullLoader)
db, fs, client = connect_to_mongodb(config_infra)

INFO:root:Successfully connected to MongoDB.


In [180]:
# Import taxonomy

taxonomy = pd.read_excel("../Categories.xlsx")

In [207]:
taxonomy

,Theme,Category,Description & Keywords,Sub-category,Description & Keywords.1
0,1: Culture,1.1: Museums,"- Description: Institutions that collect, pres...",1.1.1: Art Museums,- Description: Museums dedicated to the collec...
1,1: Culture,1.1: Museums,"- Description: Institutions that collect, pres...",1.1.2: History Museums,- Description: Museums focused on historical a...
2,1: Culture,1.1: Museums,"- Description: Institutions that collect, pres...",1.1.3: Classical Museum,- Description: Museums that focus on classical...
3,1: Culture,1.1: Museums,"- Description: Institutions that collect, pres...",1.1.4: Experience Museum,- Description: Interactive museums that offer ...
4,1: Culture,1.2: Historical Tours,- Description: Guided tours that explore histo...,1.2.1: City Walking Tours,- Description: Guided walking tours through ci...
...,...,...,...,...,...
149,5: Urban & Modern Experiences,5.5: Transportation & Travel Services,- Description: Services that facilitate travel...,5.5.2: Car Rentals,- Description: Services that offer vehicles fo...
150,5: Urban & Modern Experiences,5.5: Transportation & Travel Services,- Description: Services that facilitate travel...,5.5.3: Private Airport Transfers,- Description: Personalized transportation ser...
151,5: Urban & Modern Experiences,5.5: Transportation & Travel Services,- Description: Services that facilitate travel...,5.5.4: Rail Passes,- Description: Passes that allow unlimited tra...
152,5: Urban & Modern Experiences,5.5: Transportation & Travel Services,- Description: Services that facilitate travel...,5.5.5: Train Tickets,- Description: Tickets for travel on trains.\n...


In [212]:
df_categories = taxonomy[["Category", "Description & Keywords"]]
df_categories.drop_duplicates(inplace=True)
categories = list(df_categories["Category"])
categories = [el.split(": ")[1] for el in categories]
categories_descriptions = list(df_categories["Description & Keywords"])
categories_descriptions = [el.split("\n")[0].replace("- Description: ", "") for el in categories_descriptions]

/tmp/ipykernel_1250/2346911906.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_categories.drop_duplicates(inplace=True)


In [215]:
data = read_object(fs, "product_textual_lang_summarized")
data = pd.DataFrame(data)
data.fillna("", inplace=True)

In [216]:
batch = data[data["PRODUCTCODE"].isin(["16168P1", "9866P2", "192649P1"])]
texts = batch["pdt_product_detail_PRODUCTDESCRIPTION_SUMMARIZED"].tolist()
product_ids = batch["PRODUCTCODE"].tolist()

In [206]:
from transformers import pipeline

zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/bge-m3-zeroshot-v2.0")

for t, p in zip(texts, product_ids):

    context = "This is a tour description: "
    text = context + t
    hypothesis_template = "This tour description mentions {}"

    output = zeroshot_classifier(text, categories, hypothesis_template=hypothesis_template, multi_label=True)
    filtered_labels = [label for label, score in zip(output['labels'], output['scores']) if score > 0.9]
    print(50*"-")
    print(p)
    print(t)
    print(filtered_labels)

--------------------------------------------------
16168P1
3-hour walking tour of Barcelona's El Born and Gothic Quarter neighborhoods. Follow your local foodie guide as you visit four of the city's best tapas bars.
['Entertainment', 'Dining Experiences', 'Culinary Tours', 'Shopping and Fashion']
--------------------------------------------------
192649P1
2.5 hour tour takes you through the historic centre of Barcelona. Taste some of the city’s most famous street food at the market La Boqueria. Pass by the Gothic Quarter and El Call, the Jewish neighbourhood.
['Historical Tours', 'Dining Experiences', 'Culinary Tours']


KeyboardInterrupt: 